In [1]:
from tqdm import tqdm
import re
import pandas as pd
import os
from transliterate import translit
import pymorphy2
from pyaspeller import Word

Работаем с примерами из RLC.

In [5]:
def open_table(file):
    xl = pd.ExcelFile(file)
    df1 = xl.parse('Sheet1')
    df_rlc = df1.iloc [:, [0,3,4,5,6,8,9,10,11,12,13,14]] #Выбираем те столбцы, которые могут понадобиться.
    df_rlc = df_rlc.fillna('') #NaN заменяем '', чтобы не возникало ошибок при дальнейшей работе программы.
    return df_rlc

In [6]:
df_1_rlc = open_table('results_1_100.xlsx')

In [7]:
df_1_rlc

,Номер примера,Автор,Язык,Русский язык,Уровень,Оригинальное предложение,Исправленное предложение,Тег,Ошибка,Исправление,Разметчик,Комментарий
0,1,"Katya (F, HL, AM)",eng,HL,AM,Этот металл может сохраняться в организме чело...,Этот металл может сохраняться в организме чело...,Syntax,воздействую,воздействуя,Annotator1,
1,1,"Katya (F, HL, AM)",eng,HL,AM,Этот металл может сохраняться в организме чело...,Этот металл может сохраняться в организме чело...,"Ortho, Altern",желудожно-кишечный,желудочно-кишечный,MZambrzhitskaya,
2,2,"Katya (F, HL, AM)",eng,HL,AM,Основной путь попадания свинца в организм - {{...,Основной путь попадания свинца в организм - * ...,"Syntax, Insert, Transfer",это,,Annotator1,
3,3,"Olga (F, HL, AL)",eng,HL,AL,Стали строил монументальные здания чтобы эти з...,"*Сталин* строил монументальные *здания,* чтобы...",Ortho,Стали,Сталин,ubagryanskaya,
4,3,"Olga (F, HL, AL)",eng,HL,AL,Стали строил монументальные здания чтобы эти з...,"*Сталин* строил монументальные *здания,* чтобы...",Ref,эти здания,они,ubagryanskaya,
...,...,...,...,...,...,...,...,...,...,...,...,...
1027,425,"Anna (F, HL, AL)",eng,HL,AL,Дети привыкают к вкусу такой пищи и так же {{р...,Дети привыкают к вкусу такой пищи и так же рек...,syntax,рекламам,,admin,
1028,425,"Anna (F, HL, AL)",eng,HL,AL,Дети привыкают к вкусу такой пищи и так же {{р...,Дети привыкают к вкусу такой пищи и так же рек...,orpho,превычки,,admin,
1029,426,"Anna (F, HL, AL)",eng,HL,AL,"{{По}} {{статистике}}, {{если}} {{у}} {{одного...",По статистике если у одного родителя избыточны...,syntax,,у,admin,
1030,427,"Anna (F, HL, AL)",eng,HL,AL,{{между}} 16% и 33% детей и подростков имели и...,между 16% и 33% детей и подростков имели избыт...,syntax,между,,admin,


In [8]:
def to_str(df):
    str_for_dict = (df['Оригинальное предложение'] + ' | ' + df['Тег'] + ' | ' + df['Ошибка']+ ' | ' + df['Исправление']).to_list()
    return str_for_dict

In [9]:
list_1_rlc = to_str(df_1_rlc)

In [10]:
def dict_from_list(list_from_str):
    dict_df_rlc = {}

    for element in tqdm(list_from_str):
        element = element.split(' | ')
        sentence = element[0]
        sentence = sentence.replace('{{', '')
        sentence = sentence.replace('}}', '')
        if sentence[0] != '&':
            if sentence[0] == '.':
                sentence = sentence[1:] 
            if sentence[len(sentence)-1] != '.':
                sentence = sentence + '.'
            if '?' in sentence:
                line = sentence.split('? ')
                if len(line) > 1:
                    if line[0] not in dict_df_rlc:
                        line[0] = line[0]+'?'
                        dict_df_rlc[line[0]] = dict({'Tag': [], 'Mistake': [], 'Correct': []})
                        dict_df_rlc[line[0]]['Tag'].append(element[1])
                        dict_df_rlc[line[0]]['Mistake'].append(element[2])
                        dict_df_rlc[line[0]]['Correct'].append(element[3])
                    if line[1]not in dict_df_rlc:
                        dict_df_rlc[line[1]] = dict({'Tag': [], 'Mistake': [], 'Correct': []})
                        dict_df_rlc[line[1]]['Tag'].append(element[1])
                        dict_df_rlc[line[1]]['Mistake'].append(element[2])
                        dict_df_rlc[line[1]]['Correct'].append(element[3])
            elif sentence not in dict_df_rlc:
                dict_df_rlc[sentence] = dict({'Tag': [], 'Mistake': [], 'Correct': []})
                dict_df_rlc[sentence]['Tag'].append(element[1])
                dict_df_rlc[sentence]['Mistake'].append(element[2])
                dict_df_rlc[sentence]['Correct'].append(element[3])
    return dict_df_rlc

In [11]:
dict_df_1_rlc = dict_from_list(list_1_rlc)

100%|██████████████████████████████████████████████████████████████████████████| 1032/1032 [00:00<00:00, 262191.64it/s]


In [271]:
def to_file_for_udpipe(dict_rlc):
    with open('original_rlc.txt', 'a', encoding = 'utf-8') as f:
        for element in dict_rlc:
            f.write(element)
            f.write('\n')
    return f

In [19]:
to_file_for_udpipe(dict_df_1_rlc)

<_io.TextIOWrapper name='original_rlc.txt' mode='a' encoding='utf-8'>

Работаем с расшифровками.

In [55]:
def from_file(path_text):
    with open('all_texts.txt', 'a', encoding='utf-8') as f_main:
        for filename in os.listdir(path_text):
            with open(os.path.join(path_text, filename), 'r') as f:
                for line in f.readlines():
                    if '*CHI:' in line:
                        if '\\tab' in line:
                            line = line.split('\\tab')
                        elif '\t' in line:
                            line = line.split('\t')
                        line_new = line[1]
                        line_new = line_new[:-2]
                        f_main.write(line_new)
                        f_main.write('\n')
    return f_main

In [56]:
from_file('texts_transl')

<_io.TextIOWrapper name='all_texts.txt' mode='a' encoding='utf-8'>

In [57]:
def translit_to_rus(file):
    lines = []
    with open('all_texts.txt', 'r', encoding='utf-8') as f:
        with open('translit.txt', 'a', encoding='utf-8') as f2:  
            for line in f.readlines():
                ru_text = translit(line, 'ru')
                ru_text = ru_text.replace('@', '')
                ru_text = ru_text.replace('[', '')
                ru_text = ru_text.replace(']', '')
                ru_text = ru_text.replace('+/', '')
                ru_text = ru_text.replace('/', '')
                ru_text = ru_text.replace('#', '')
                ru_text = ru_text.replace('=', '')
                ru_text = ru_text.replace('&', '')
                ru_text = ru_text.replace('2', '')
                ru_text = ru_text.replace('0', '')
                ru_text = ru_text.replace('е~', 'э')
                ru_text = ru_text.replace('Е~', 'Э')
                if ru_text[-2] != '.' and ru_text[-2] != '?':
                    ru_text = ru_text.replace(ru_text[-1], '.\n')
                if len(ru_text.split()) > 3:
                    if 'xxx' not in ru_text:
                        if ru_text not in lines:
                            f2.write(ru_text)
                            lines.append(ru_text)
    return f2

In [58]:
translit_to_rus('all_texts.txt')

<_io.TextIOWrapper name='translit.txt' mode='a' encoding='utf-8'>

Расшифровки без транслитерации

In [47]:
def txt_rus(path_text):
    lines = []
    with open('txt_rus.txt', 'a', encoding='utf-8') as f_main:
        for filename in os.listdir(path_text):
            with open(os.path.join(path_text, filename), 'r', encoding='utf-16-le') as f:
                for line in f.readlines():
                    if '*CHI:' in line:
                        line = line.split('\t')
                        line_new = line[1]
                        line_new = line_new.replace('&', '')
                        line_new = line_new.replace('@', '')
                        line_new = line_new.replace('[', '')
                        line_new = line_new.replace(']', '')
                        line_new = line_new.replace('2', '')
                        line_new = line_new.replace('+/', '')
                        line_new = line_new.replace('#', '')
                        line_new = line_new.replace('(', '')
                        line_new = line_new.replace(')', '')
                        line_new = line_new.replace('2', '')
                        line_new = line_new.replace('/', '')
                        line_new = line_new.replace('errv-', '')
                        if len(line_new.split()) > 3:
                            if ('xxx' not in line_new) and ('ххх' not in line_new) and ('=' not in line_new):
                                if line_new not in lines:
                                    f_main.write(line_new)
                                    lines.append(line_new)
    return f_main

In [48]:
txt_rus('texts')

<_io.TextIOWrapper name='txt_rus.txt' mode='a' encoding='utf-8'>

------------------------------------------------------------------

Работа с примерами из RLC

In [427]:
#dict_df_1_rlc #словарь с тэгами и ошибками

Его руками пришлось подправить, так как UDPipe в самом конце разбил одну строчку на 2 предложения, а в другом случае не сделал этого.

In [298]:
dict_df_1_rlc['По статистике, если у одного родителя избыточный вес, у их ребенка возможность ожирения уже 50%. между 16% и 33% детей и подростков имели избыточный вес.'] = {'Tag': ['Syntax', 'syntax'],
  'Mistake': ['', 'между'],
  'Correct': ['у ', '']}
del dict_df_1_rlc['По статистике, если у одного родителя избыточный вес, у их ребенка возможность ожирения уже 50%.']
del dict_df_1_rlc['между 16% и 33% детей и подростков имели избыточный вес.']
del dict_df_1_rlc['Две главные причины: расширение сфер влияния СССР доктрина сдерживания социализма).']
dict_df_1_rlc['Две главные причины: расширение сфер влияния СССР доктрина сдерживания социализма).'] = {'Tag': ['Syntax'],
  'Mistake': ['.'],
  'Correct': [' и']}

Функция для прочтения conllu

In [2]:
def read_conllu(file):
    sent_list = []
    with open(file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        sent_lines = []
        for line in lines:
            line = line.strip()
            if not line:
                if sent_lines:
                    sent_list.append(sent_lines)
                    sent_lines = []
            else:
                sent_lines.append(line)
        if sent_lines:
            sent_list.append(sent_lines)
    return sent_list


Функция для записи conllu

In [369]:
def write_conllu(sent_list, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        for sent in sent_list:
            f.write(sent)
            f.write('\n')

In [3]:
sent_list = read_conllu('1_rlc.conllu')

Стараемся перенести почти все исправленные ошибки в MISC

In [12]:
# проход по предложениям в файле и сравнение со словарем
flag = False
sent_new = []
not_in_file = []

for sent in sent_list:
    sent_text = ''
    for line in sent:
        if '# text' in line:
            sent_full = line[9:]
            sent_new.append(line)
        elif '#' not in line:
            line_word = line.split('\t')
            # извлекаем слово из строки
            word = line_word[1]
            # проверяем, есть ли слово в списке ошибок для данного предложения
            try: 
                for i in range(len(dict_df_1_rlc[sent_full]['Mistake'])):
                    mistake = dict_df_1_rlc[sent_full]['Mistake'][i]
                    tag = dict_df_1_rlc[sent_full]['Tag']
                    tag = tag[0].split(', ')
                    if mistake == word:
                        for t in range(len(tag)):
                            if tag[t] != 'Insert' and tag[t] not in line_word[9]:
                                if str(dict_df_1_rlc[sent_full]['Correct'][i]) != ' ':
                                    line_word[9] = ''.join([tag[t], '=', str(dict_df_1_rlc[sent_full]['Correct'][i])])
                                else:
                                    line_word[9] = ''.join(tag[t])
                                x = t+1
                                while x != len(tag):
                                    line_word[9] = line_word[9]+'|'+ tag[x]
                                    x+=1
                        line_new = '\t'.join(line_word)
                        sent_new.append(line_new)
                    else:
                         sent_new.append(line)              
            except KeyError:
                sent_new.append(line)
                not_in_file.append(line)
        else:
            sent_new.append('\n')
            sent_new.append(line)
            not_in_file.append(line)

In [426]:
write_conllu(sent_new, 'sentences_rlc_meta.conllu')

Посмотрим на те метки, которые не попали в файл

In [31]:
#not_in_file

In [26]:
check = []

for i in range(len(not_in_file)-1):
    if '#' in not_in_file[i] and '1\t' in not_in_file[i+1]:
        check.append(not_in_file[i])

Попытка найти ошибки в расшифровке

In [19]:
morph = pymorphy2.MorphAnalyzer()
spell = pyaspeller.YandexSpeller()

In [21]:
mistake_dict = {}

with open('txt_rus.txt', 'r', encoding='utf-8') as f:
    text = f.read()

words = text.split()
for word in tqdm(words):
    check = Word(word)
    if check.correct == False:
        mistake_dict[word] = check.variants

100%|██████████████████████████████████████████████████████████████████████████████| 1486/1486 [02:20<00:00, 10.57it/s]


In [22]:
mistake_dict

{'потерятся,': ['потеряться', 'потерялся', 'потеряется', 'потеряются'],
 'побежать.': ['побеждать', 'пробежать', 'побегать']}

In [34]:
mistake_dict_translit = {}

with open('translit.txt', 'r', encoding='utf-8') as f:
    text = f.read()

words = text.split()
for word in tqdm(words):
    check = Word(word)
    if check.correct == False:
        mistake_dict_translit[word] = check.variants

100%|██████████████████████████████████████████████████████████████████████████████| 2660/2660 [03:54<00:00, 11.32it/s]


In [36]:
#mistake_dict_translit

------------------------------------------------------------------------------

Записываем всё в один файл

In [72]:
folder_path = 'file_conllu'

sentence_number = 1

with open('Heritage_rus_corpus_2.conllu', 'a', encoding='utf-8') as f:
    for file_name in os.listdir(folder_path): 
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as f2:
            for line in f2:
                if line.startswith(''# sent_id =''):
                    line = '# sent_id = ' + str(sentence_number)
                    sentence_number += 1
                if line[-1]!='\n':
                    f.write(line)
                    f.write('\n')
                else:
                    f.write(line)

In [12]:
with open('Heritage_rus_corpus.conllu', 'a', encoding='utf-8') as f2:
    with open('Heritage_rus_corpus_2.conllu', 'r', encoding='utf-8') as f:
        last_line = ''
        for line in f:
            if '# sent_id' in last_line:
                if int(last_line[12:]) < 410:
                    f2.write('# Metadata = студент | письменная речь | литературный стиль | университет')
                    f2.write('\n')
                else:
                    f2.write('# Metadata = ребёнок дошкольного возраста | устная речь | разговорный стиль | дом')
                    f2.write('\n')
            if last_line == '\n' and line =='\n':
                line = ''
            f2.write(line)
            last_line = line